In [58]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow.keras.utils import image_dataset_from_directory


## 🧹 Data Preparation

In [59]:
training_set = image_dataset_from_directory(
"./train", #Path to the training data directory
labels="inferred",
label_mode="categorical",
class_names=None,
color_mode="rgb",
batch_size=16,
image_size=(224, 224),
shuffle=True,
seed=None,
validation_split=None,
subset=None,
interpolation="bilinear",
follow_links=False,
crop_to_aspect_ratio=False,
)

Found 76515 files belonging to 4 classes.


In [13]:
validation_set = image_dataset_from_directory(
"./val", #Path to the training data directory
labels="inferred",
label_mode="categorical",
class_names=None,
color_mode="rgb",
batch_size=16,
image_size=(224, 224),
shuffle=True,
seed=None,
validation_split=None,
subset=None,
interpolation="bilinear",
follow_links=False,
crop_to_aspect_ratio=False,
)

Found 21861 files belonging to 4 classes.


In [14]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [15]:
INPUT_SHAPE = (224, 224, 3)

In [16]:
mobnet = tf.keras.applications.MobileNetV3Large(
input_shape=INPUT_SHAPE,
alpha=1.0,
minimalistic=False,
include_top=True,
weights="imagenet",
input_tensor=None,
classes=1000,
pooling=None,
dropout_rate=0.2,
classifier_activation="softmax",
include_preprocessing=True,
# name="MobileNetV3Large",
)

In [49]:
# if u want to perform transfer learning then execute the below line
#mobnet.trainable = False #Freezing the weights of the pre-trained model

In [17]:
model = tf.keras.Sequential()

In [18]:
model.add(tf.keras.Input(shape=INPUT_SHAPE))

In [19]:
model.add(mobnet)

In [20]:
model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

In [21]:
# created a custom f1 score using function
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

metrics_list = ["accuracy", f1_score]

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=metrics_list)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 1000)             5507432   
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4004      
                                                                 
Total params: 5,511,436
Trainable params: 5,487,036
Non-trainable params: 24,400
_________________________________________________________________


In [17]:
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=15)

Epoch 1/15
4783/4783 [==============================] - 740s 151ms/step - loss: 1.0165 - accuracy: 0.8751 - f1_score: 0.0702 - val_loss: 0.7625 - val_accuracy: 0.8954 - val_f1_score: 0.4786
Epoch 2/15
4783/4783 [==============================] - 704s 147ms/step - loss: 0.6081 - accuracy: 0.9114 - f1_score: 0.4795 - val_loss: 0.4748 - val_accuracy: 0.9511 - val_f1_score: 0.4818
Epoch 3/15
4783/4783 [==============================] - 718s 150ms/step - loss: 0.3933 - accuracy: 0.9543 - f1_score: 0.6229 - val_loss: 0.3153 - val_accuracy: 0.9618 - val_f1_score: 0.6796
Epoch 4/15
4783/4783 [==============================] - 715s 149ms/step - loss: 0.2710 - accuracy: 0.9606 - f1_score: 0.7919 - val_loss: 0.2849 - val_accuracy: 0.9371 - val_f1_score: 0.7925
Epoch 5/15
4783/4783 [==============================] - 720s 151ms/step - loss: 0.1929 - accuracy: 0.9680 - f1_score: 0.8416 - val_loss: 0.1699 - val_accuracy: 0.9683 - val_f1_score: 0.8463
Epoch 6/15
4783/4783 [============================

In [45]:
model.save("./Trained_Eye_disease_model.h5")



In [49]:
import tensorflow as tf

model = tf.keras.models.load_model(
    "Trained_Eye_disease_model.keras",
    compile=False
)

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 1000)             5507432   
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4004      
                                                                 
Total params: 5,511,436
Trainable params: 5,487,036
Non-trainable params: 24,400
_________________________________________________________________


In [56]:
import os
print(os.path.getsize("Training_history.pkl"))

0


In [57]:
training_history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=15
)

import pickle
with open("Training_history.pkl", "wb") as f:
    pickle.dump(training_history.history, f)

import os
print("New pkl size:", os.path.getsize("Training_history.pkl"))


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [55]:
training_history.history

NameError: name 'training_history' is not defined

In [ ]:
import pickle
with open("Training_history.pkl", "rb") as f:
    load_history = pickle.load(f)


In [53]:
#SAVING HISTORY 
import pickle
with open("Training_history.pkl", "wb") as f:
    pickle.dump(training_history.history, f)

NameError: name 'training_history' is not defined

In [54]:
# reloading history
import pickle   
with open("Training_history.pkl", "rb") as f:
   load_history = pickle.load(f)
   print(load_history)

EOFError: Ran out of input

In [4]:
#load history key
load_history.keys()

NameError: name 'load_history' is not defined

In [3]:
#loss visualization
epochs = [i for i in range(1,16)]
plt.plot(epochs, load_history['loss'], color='red', label='Training Loss')
plt.plot(epochs, load_history['val_loss'], color='blue', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'load_history' is not defined

## Model Evaluation

In [11]:
# load model with the custom f1_score metric defined earlier
model = tf.keras.models.load_model("./Trained_Eye_disease_model.h5", custom_objects={"f1_score": f1_score})

NameError: name 'f1_score' is not defined

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 1000)             5507432   
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4004      
                                                                 
Total params: 5,511,436
Trainable params: 5,487,036
Non-trainable params: 24,400
_________________________________________________________________


In [ ]:
test_set = image_dataset_from_directory(
"./test", #Path to the training data directory
labels="inferred",
label_mode="categorical",
class_names=None,
color_mode="rgb",
batch_size=16,
image_size=(224, 224),
shuffle=False,
seed=None,
validation_split=None,
subset=None,
interpolation="bilinear",
follow_links=False,
crop_to_aspect_ratio=False,
)

Found 10933 files belonging to 4 classes.


In [44]:
test_loss,test_acc,f1_score = model.evaluate(test_set)

684/684 [==============================] - 31s 45ms/step - loss: 0.1217 - accuracy: 0.9713 - f1_score: 0.2469


In [35]:
test_loss,test_acc,f1_score

(0.12166132777929306, 0.9712796211242676, 0.8409831523895264)

In [47]:
#computing true labels from test set
true_categories = tf.concat([y for x, y in test_set], axis=0)

Y_true = tf.argmax(true_categories, axis=1)

In [39]:

true_categories

<tf.Tensor: shape=(10933, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)>

In [49]:
Y_true

<tf.Tensor: shape=(10933,), dtype=int64, numpy=array([0, 0, 0, ..., 3, 3, 3], dtype=int64)>

In [2]:
#computed predicted labels from test set
predicted_categories = model.predict(test_set)
predicted_categories

NameError: name 'model' is not defined

In [51]:
Y_Pred = tf.argmax(predicted_categories, axis=1)
Y_Pred

<tf.Tensor: shape=(10933,), dtype=int64, numpy=array([0, 0, 0, ..., 3, 3, 3], dtype=int64)>

## 1 classification report

In [10]:
from sklearn.metrics import classification_report
print(classification_report(Y_true, Y_Pred))

NameError: name 'Y_true' is not defined

## 2 Confusion Matrix


In [4]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_true, Y_Pred)
cm

NameError: name 'Y_true' is not defined

In [3]:
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, annot_kws={"size": 8})
plt.xlabel('Predicted Labels', fontsize=10)
plt.ylabel('Actual Class', fontsize=10)
plt.title('Human Eye Disease Prediction Confusion Matrix',fontsize=12)
plt.show()


NameError: name 'cm' is not defined

<Figure size 1000x1000 with 0 Axes>